In [1]:
from Triple import Triple
from tqdm import tqdm
from spacy.matcher import Matcher
from data import psychology_data
import networkx as nx
import matplotlib.pyplot as plt
import spacy
import scispacy
import re
import pandas as pd
pd.set_option('display.max_colwidth', 200)
%matplotlib qt
nlp = spacy.load('en_core_web_lg')

Psychology corpus 886 sentences


In [3]:
def knowledge_graph(triples):
    G = nx.Graph()
    for triple in triples:
        G.add_node(triple[0])
        G.add_node(triple[1])
        G.add_node(triple[2])
        G.add_edge(triple[0], triple[1])
        G.add_edge(triple[1], triple[2])

    pos = nx.spring_layout(G)
    plt.figure()
    nx.draw(G, pos, edge_color='black', width=1, linewidths=1,
            node_size=500, node_color='#09a3d5', alpha=0.9,
            labels={node: node for node in G.nodes()})
    plt.axis('off')
    plt.show()

In [4]:
triples = []

for i, sent in enumerate(tqdm(psychology_data['sentence'])):
    triples.append(Triple(sent).get_triple())

#   # Graphs (uncomment to create artefacts)
    # Triple(sent).tree('tree1'+str(i))
    # Triple(sent).graph('tree1'+str(i))

# # Knowleadge Graph
knowledge_graph(triples)



100%|██████████| 886/886 [00:04<00:00, 186.09it/s]


In [ ]:

# percy = "Percy the mockingbird spent the whole warm season chirping and twittering"
